In [6]:
import torch
import numpy as np
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from torch import nn, optim
from torchvision.datasets import MNIST

import matplotlib.pyplot as plt


# 载入训练集数据，同时把数据转换为tensor，同时下载数据
train_dataset = datasets.MNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)

# 载入测试集数据，同时把数据转换为tensor，同时下载数据
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.ToTensor(), download=True)

# 文件分别为训练集测试集他们分别的数据和标签
# 批次大小
batch_size = 64
# 装载数据集,dataloader为数据的装载器，数据来源于dataset=train_dataset, 大小为batch_size=batch_size，方式为随机打乱
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=8)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True, pin_memory=True, num_workers=8)
# 展示数据
for i, data in enumerate(train_loader):
    inputs, labels = data
    print(inputs.shape)
    print(labels.shape)
    break
Dop = 0.2  # 0.1的神经元不工作


class LSTM(nn.Module):
    # 初始化，定义网络结构
    def __init__(self):
        # 初始化nnModule
        super(LSTM, self).__init__()
        # input：输入特征的大小
        # hidden：LSTM模块的数量
        # layers：隐藏层的层数
        # lstm默认input(seq_len, batch, feature)
        # batchfirst后，变成(feature, seq_len, batch)
        self.lstm = nn.LSTM(
            input_size=28,
            hidden_size=64,
            num_layers=1,
            batch_first=True
        )
        self.fc = nn.Linear(64, 10)
        self.softmax = nn.Softmax(dim=1)

    # 前向计算，定义网络计算
    def forward(self, x):
        x = x.view(-1, 28, 28)
        # 三个输出
        # output:[batch,seq_len,hidden_size]，包含每个序列的输出结果
        # seq_len取27时，就是最后一个结果
        # 虽然LSTW的batch_first为True，但是h_n,c_n的第0个维度还是numlayers
        # h_n:[num_layers, batch, hidden_size]，只包含最后一个序列的输出结果hidden
        # c_n:[num_layers, batch, hidden_size]，只包含最后一个序列的输出结果cell
        output, (h_n, c_n) = self.lstm(x)
        #output_last = self.fc(output[:, -1, :])  #只要最后一个结果
        output_last = h_n[-1, :, :]  #只要最后一个结果
        x = self.fc(output_last)
        outs = self.softmax(x)

        return outs


LR = 0.001
# 定义模型
model = LSTM()

# 载入模型参数
model.load_state_dict(torch.load('modle/LSTMnum.pth'))
# 定义损失函数,交叉熵损失
cross_entropy = nn.CrossEntropyLoss()




def test():
    # 测试状态
    model.eval()
    # 计算测试集的准确率
    correct = 0
    for i, data in enumerate(test_loader):
        inputs, labels = data

        out = model(inputs)
        # 计算out中，最大值所在位置
        _, predicted = torch.max(out.data, 1)

        correct += (predicted == labels).sum()
    print("Test acc:{0}".format(correct.item() / len(test_loader.dataset)))

    # 计算训练集的准确率
    correct = 0
    for i, data in enumerate(train_loader):
        inputs, labels = data
        out = model(inputs)
        # 计算out中，最大值所在位置
        _, predicted = torch.max(out.data, 1)

        correct += (predicted == labels).sum()
    print("Train acc:{0}".format(correct.item() / len(train_loader.dataset)))


    # 不再训练
    # train()


test()



torch.Size([64, 1, 28, 28])
torch.Size([64])


C:\Users\BLKDASH\AppData\Local\Temp\ipykernel_17576\1526851729.py:74: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('modle/LSTMnum.pth'))


Test acc:0.9638
Train acc:0.9645333333333334
